In [1]:
import pandas as pd
from collections import defaultdict, OrderedDict, ChainMap, Counter
from efficient_apriori import apriori
import itertools
import json
from os import path

----------------
# Process Data
--------------------
### saves to output file and uses that instead.

In [2]:
transactions = []
if path.exists('preprocessedTrans.txt'):
    fi = open('preprocessedTrans.txt','rt')
    line = fi.readlines()
    for l in line:
        transactions.append(tuple(l.split()))
    fi.close()
else: #open raw data and process it
    fi = open('BMS2.txt','rt')
    fout = open('preprocessedTrans.txt','wt')
    lines = fi.readlines()
    for l in lines:
        liner = l.replace('-1 -2','') #remove ending
        liner = liner.replace(' -1','') #replace -1 with comma
        fout.write(liner)
        transactions.append(tuple(liner.split()))
    fi.close()
    fout.close()


-----------------------------
# Run Apriori
-------------------------
###   return frequent itemsets

In [5]:
%%time
itemsets,rules = apriori(transactions,min_support=0.005,min_confidence=0.7)
    

Wall time: 2min 7s


In [6]:
itemsets

{1: {('55455',): 598,
  ('55879',): 394,
  ('55835',): 953,
  ('55831',): 1291,
  ('55403',): 416,
  ('55839',): 857,
  ('55407',): 532,
  ('56765',): 950,
  ('55435',): 461,
  ('55863',): 635,
  ('55411',): 416,
  ('55287',): 1477,
  ('55323',): 3417,
  ('55295',): 1834,
  ('55331',): 775,
  ('55327',): 2150,
  ('82875',): 396,
  ('89849',): 729,
  ('89845',): 544,
  ('55283',): 2093,
  ('89453',): 637,
  ('84731',): 1226,
  ('55779',): 400,
  ('228795',): 395,
  ('88683',): 491,
  ('55343',): 1293,
  ('55859',): 1027,
  ('89705',): 907,
  ('89701',): 409,
  ('55319',): 2497,
  ('55367',): 1296,
  ('82719',): 1255,
  ('55267',): 3766,
  ('56037',): 799,
  ('55487',): 869,
  ('55883',): 745,
  ('55351',): 2249,
  ('55551',): 1423,
  ('55527',): 468,
  ('55271',): 2284,
  ('55703',): 444,
  ('55483',): 539,
  ('55463',): 1046,
  ('55291',): 2003,
  ('56761',): 2048,
  ('55819',): 694,
  ('222643',): 452,
  ('55315',): 2248,
  ('56041',): 960,
  ('55559',): 1009,
  ('55875',): 1331,
  ('

------------
# Get unique set of Support Values
--------------

In [8]:
def getSuppK(itemlist):
    skeys = []
    for k,v in itemlist.items():
        for a,s in v.items():
            skeys.append(s)
    check = Counter(skeys)
    return set(skeys),check

------------------------------------------------------------------
# Equi Support Function 2
----------------------------------------------------------------
Issue here is that it inserts duplicates of the same tuple

In [11]:
######### Equisupport group with no libraries. No preporcessing of the result of Apriori()
def getEquiSupp(itemlist):
    equiDict = {}
    skeys,_ = getSuppK(itemlist) # returns a set of unique support values as keys
    for key,val in itemlist.items():
        for item,supp in val.items():
            if supp in skeys:
                equiDict.setdefault(supp,[]).append((item,supp))
    return equiDict,skeys
equal_groups,s_key = newequigrp(itemsets)

----------------------------------------------
# Checks if Frequent Itemsets are closed,
##  Returns the closed itemsets and the left over candidates
-------------------------------------------------

In [14]:
def getClosedSet(equilistgrp):
    """ get closed set """
    candidates = []
    closed = []
    for i in equilistgrp:
        size = len(equilistgrp[i])
        for a,b in zip(equilistgrp[i],equilistgrp[i]):
            if a[0] == b[0]:
                closed.append(a)
            # print("here is a -->",a[0])
            if frozenset(a[0]).issubset(b[0]):
                # equilistgrp[i].remove(a)
                candidates.append(a)
                # print(a, " is Not closed")
            if frozenset(a[0]).issuperset(b[0]):
                # print(b," is Not closed")
                candidates.append(b)
            else:
                # print(a, "is closed")
                closed.append(a)
    return candidates, closed    
o, c = getClosedSet(equal_groups)
#currently qworking no changes reqd

--------------------------------------
# Function to convert lists to the expected Output
-----------------------------------------------

In [17]:
#store result as dict
def formatOutputDict(ret_list):
    res_dict = {1:{},2:{},3:{},4:{},5:{},6:{},7:{}}
    # print("llen of list ",len(ret_list))
    for i in ret_list:
        # print("here is {}",i)
        length = len(i[0])
        if length in res_dict.keys():
            # print(closeddict[length])
            dictval = {i[0]:i[1]} 
            # print(dictval)
            res_dict[length][i[0]] = i[1]
    return res_dict

------------------------
# Return Maximal Itemsets
------------------------
### checks closed set for maximal items. using pairwise comparison. Only checks once, so in test when I added the same tuple multiple times it would only remove one

In [213]:
def maxSet(closed_items):
    cl_cpy = closed_items.copy()
    dropd_items = [] #items that aren't maximal
    maximal = []
    print("len of list before",len(cl_cpy)) #see how many are dropped
    for i in closed_items:
        for j in cl_cpy:
            if i != j:
                if frozenset(j[0]).issubset(i[0]): 
                    dropd_items.append(j)
                    # cl_cpy.remove(j)
                else:
                    print("found maximal",j)
                    maximal.append(j)
    print("end length",len(maximal))
    # print("DROP LIST",dropd_items)
    return cl_cpy, dropd_items
maxSet(c)


Error: AssertionError

In [206]:
c

[(('55455',), 598),
 (('55267', '55283'), 598),
 (('55291', '55295', '55323'), 598),
 (('55839',), 857),
 (('55879',), 394),
 (('84743',), 394),
 (('55543', '55547'), 394),
 (('55435',), 461),
 (('55555', '55559'), 461),
 (('55835',), 953),
 (('56765',), 950),
 (('55831',), 1291),
 (('55407',), 532),
 (('55863',), 635),
 (('55403',), 416),
 (('55411',), 416),
 (('55267', '55295', '55315', '55319', '55323'), 416),
 (('55287',), 1477),
 (('55327',), 2150),
 (('55323',), 3417),
 (('55295',), 1834),
 (('55331',), 775),
 (('82875',), 396),
 (('55287', '55295', '55319', '55323'), 396),
 (('55295', '55319', '55323', '55351'), 396),
 (('89849',), 729),
 (('89845',), 544),
 (('55271', '55283'), 544),
 (('89453',), 637),
 (('55283',), 2093),
 (('84731',), 1226),
 (('55779',), 400),
 (('83727',), 400),
 (('250515',), 400),
 (('55271', '55335'), 400),
 (('228795',), 395),
 (('88683',), 491),
 (('89717',), 491),
 (('55343',), 1293),
 (('55859',), 1027),
 (('89701',), 409),
 (('253633', '55367'), 40

In [976]:
%%time
itemsets,rules = apriori(transactions,min_support=0.005,min_confidence=0.7)

Wall time: 5min 11s


In [200]:
test = []
totalItems = 0
for k,v  in itemsets.items():
    print('***************************************************\n')
    test.append({value:key for key,value in v.items()})
    totalItems +=len(v)
    print("for index {}, the number of  frequent sets is {}\n".format(k,len(v)))
print("Total Num of Freq Itemsets is:",totalItems)

***************************************************

for index 1, the number of  frequent sets is 170

***************************************************

for index 2, the number of  frequent sets is 120

***************************************************

for index 3, the number of  frequent sets is 88

***************************************************

for index 4, the number of  frequent sets is 28

***************************************************

for index 5, the number of  frequent sets is 2

Total Num of Freq Itemsets is: 408


In [ ]:
#maximal set
#set is maximal if any of its supersets have a lower support
# take the closed set, get largest items set and remove all subsets
maxSet = {1:{},2:{},3:{},4:{},5:{},6:{},7:{}}
def maximalSet(closedset):
    maxCandidates =
    pass

In [973]:
maxSet(scl)

len of list before 39
end length 35
DROP LIST [((55267,), 15), ((55295, 55351), 1), ((222339,), 1), ((55267, 222339), 5)]


([((55267, 55271, 55295, 55351), 10),
  ((55267, 55271, 55287, 55351), 10),
  ((55287, 55295), 10),
  ((55295, 55323), 10),
  ((55267, 55315), 10),
  ((55271, 222331), 10),
  ((55271, 55283), 10),
  ((55271, 55323), 10),
  ((55267, 56761), 11),
  ((55351, 222339), 11),
  ((55323, 55351), 11),
  ((55839,), 10),
  ((55895,), 10),
  ((55403,), 10),
  ((55559,), 10),
  ((222459,), 10),
  ((55487,), 10),
  ((56037,), 10),
  ((55551,), 10),
  ((222471,), 10),
  ((222335,), 10),
  ((84727,), 10),
  ((55535,), 10),
  ((222319,), 10),
  ((84159,), 10),
  ((78687,), 10),
  ((55859,), 11),
  ((55483,), 11),
  ((222395,), 11),
  ((197025,), 11),
  ((55555,), 11),
  ((222351,), 11),
  ((84839,), 11),
  ((55267, 222339, 55271), 10),
  ((55267, 222339), 5)],
 [((55267,), 15), ((55295, 55351), 1), ((222339,), 1), ((55267, 222339), 5)])

In [1027]:
# equiSupport(closeddict,k_fun)
sortedFreqSet

[((55307, 55315, 55319, 55323), 388),
 ((55287, 55291, 55323), 388),
 ((229179,), 388),
 ((55271, 55323, 55351), 389),
 ((55995,), 389),
 ((55267, 55295, 55351), 390),
 ((86071,), 390),
 ((55287, 55295, 55315, 55319), 391),
 ((55287, 55291, 55319), 391),
 ((55291, 55335), 391),
 ((55295, 55307), 391),
 ((250487,), 391),
 ((55339, 55343), 392),
 ((55267, 55315, 55319, 55351), 393),
 ((55267, 55271, 55315, 55323), 393),
 ((55543, 55547), 394),
 ((55879,), 394),
 ((84743,), 394),
 ((228795,), 395),
 ((55287, 55295, 55319, 55323), 396),
 ((55295, 55319, 55323, 55351), 396),
 ((82875,), 396),
 ((55287, 55319, 55323, 55351), 397),
 ((55319, 55323, 55343), 397),
 ((55543, 55555), 397),
 ((306069,), 397),
 ((55291, 55367), 398),
 ((56073,), 398),
 ((55271, 55335), 400),
 ((55779,), 400),
 ((83727,), 400),
 ((250515,), 400),
 ((55267, 55287, 55295), 401),
 ((55551, 55563), 401),
 ((55267, 253633), 401),
 ((55271, 55319, 55351), 402),
 ((55547, 55555), 402),
 ((55291, 55295, 55315, 55319, 55323)

In [1024]:
#sorted by support number NOT NEEDED
chmap = ChainMap(testSet[1],testSet[2],testSet[3],testSet[4],testSet[5])
sortedFreqSet = sorted(chmap.items(),key=lambda x: x[1])
activeSortedSet = dict(sortedFreqSet)

In [396]:
uniqueKeys = set(activeSortedSet.values()) #need this
freqitems = activeSortedSet.keys()

In [705]:
b = equiSupport(testSet,k_fun)

In [711]:
#need this to create my equisupport groups
equiSuppGrp = []
a = []
for k,g in itertools.groupby(sortedFreqSet,lambda x: x[1]):
    equiSuppGrp.append(list(g)) 

In [759]:
testgrp = equiSuppGrp[5:7]
len(testgrp[1])

13

In [521]:
uniqueKeys
testgrp #17,19,21,20,23
for i in range(len(testgrp)):
    print("for support ", testgrp[i][0][1], "itemset is", testgrp[i],"\n")
    print("----------------")
    for j in range(len(testgrp[i])):
        print("testgrp set: ",testgrp[i][j])

for support  17 itemset is [((55831, 55835), 17), ((56765,), 17), ((88683,), 17), ((55315,), 17), ((55855,), 17), ((203733,), 17)] 

----------------
testgrp set:  ((55831, 55835), 17)
testgrp set:  ((56765,), 17)
testgrp set:  ((88683,), 17)
testgrp set:  ((55315,), 17)
testgrp set:  ((55855,), 17)
testgrp set:  ((203733,), 17)
for support  19 itemset is [((55327,), 19), ((55319,), 19), ((222339,), 19), ((83547,), 19), ((55891,), 19)] 

----------------
testgrp set:  ((55327,), 19)
testgrp set:  ((55319,), 19)
testgrp set:  ((222339,), 19)
testgrp set:  ((83547,), 19)
testgrp set:  ((55891,), 19)
for support  20 itemset is [((55271, 55351), 20)] 

----------------
testgrp set:  ((55271, 55351), 20)
for support  21 itemset is [((82719,), 21), ((55291,), 21), ((55899,), 21)] 

----------------
testgrp set:  ((82719,), 21)
testgrp set:  ((55291,), 21)
testgrp set:  ((55899,), 21)
for support  22 itemset is [((55267, 55271), 22), ((55871,), 22)] 

----------------
testgrp set:  ((55267, 5

In [731]:
subres, clres = closedSet(testgrp,uniqueKeys)
clres

[((55267, 55271, 55295, 55351), 10),
 ((55267, 55271, 55287, 55351), 10),
 ((55271, 55287, 55351), 10),
 ((55287, 55295), 10),
 ((55295, 55323), 10),
 ((55267, 55315), 10),
 ((55267, 222339), 10),
 ((55271, 222331), 10),
 ((55271, 55283), 10),
 ((55271, 55323), 10),
 ((55839,), 10),
 ((55895,), 10),
 ((55403,), 10),
 ((55559,), 10),
 ((222459,), 10),
 ((55487,), 10),
 ((56037,), 10),
 ((55551,), 10),
 ((222471,), 10),
 ((222335,), 10),
 ((84727,), 10),
 ((55535,), 10),
 ((222319,), 10),
 ((222339,), 10),
 ((55267, 55271, 55287), 11),
 ((55267, 55287, 55351), 11),
 ((55267, 56761), 11),
 ((55295, 55351), 11),
 ((55351, 222339), 11),
 ((55323, 55351), 11),
 ((55859,), 11),
 ((55483,), 11),
 ((222395,), 11),
 ((197025,), 11),
 ((55555,), 11),
 ((222351,), 11),
 ((84839,), 11),
 ((55267, 55271, 55295), 12),
 ((55323, 55327), 12),
 ((55271, 222339), 12),
 ((55319, 55351), 12),
 ((55271, 55287), 12),
 ((55863,), 12),
 ((222323,), 12),
 ((55343,), 12),
 ((55367,), 12),
 ((222615,), 12),
 ((55

In [1124]:
# working attempt for closed set. 

subsettest = []
closedtest = []
for key in uniqueKeys:
    equiSupport = []
    for i in range(len(equiGrp)):
        if(equiGrp[i][0][1] == key):
            print("for key -->", key,"the equisupport group is: ",equiGrp[i][0])
            print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n")
            lrgfreqset = equiGrp[i][0][0]
            closedtest.append(equiGrp[i][0])
            print("arge freq set: ",lrgfreqset)
            for j in range(len(equiGrp[i])):
                if equiGrp[i][j][0] != lrgfreqset:
                    print("testgrp[{}][{}][0]: {}".format(i,j,equiGrp[i][j][0]))
                    print("this is the item set to cmp ",equiGrp[i][j][0])
                    if(frozenset(equiGrp[i][j][0]).issubset(lrgfreqset)):
                        subsettest.append(equiGrp[i][j])
                    else:
                        closedtest.append(equiGrp[i][j])
                        
    #         sameSup.append(item) #now have all same support items together



In [1299]:
from collections import Counter
kk = getSuppK(itemsets)
Counter(kk)

Counter({598: 3,
         461: 2,
         1291: 1,
         953: 1,
         857: 1,
         635: 1,
         950: 1,
         394: 3,
         416: 3,
         532: 1,
         775: 1,
         1477: 1,
         3417: 1,
         2150: 1,
         1834: 1,
         396: 3,
         544: 2,
         729: 1,
         637: 1,
         2093: 1,
         1226: 1,
         400: 4,
         395: 1,
         491: 2,
         1293: 1,
         1027: 1,
         409: 6,
         907: 1,
         2497: 1,
         1296: 1,
         1255: 1,
         1009: 1,
         694: 1,
         2248: 1,
         444: 2,
         1046: 2,
         452: 2,
         2249: 1,
         2048: 1,
         539: 1,
         869: 1,
         1331: 1,
         745: 2,
         722: 1,
         3766: 1,
         799: 1,
         2284: 1,
         468: 1,
         960: 2,
         2003: 1,
         1423: 1,
         1031: 1,
         758: 1,
         840: 1,
         1125: 1,
         407: 4,
         940: 1,
       

In [1325]:
#closed set as a function
def getSuppK(itemlist):
    skeys = []
    for k,v in itemlist.items():
        for a,s in v.items():
            skeys.append(s)
            # if s in skeys:
            #     dt[s] = v.items()
    return skeys

def closedSet(equilistgrp,supportkeys): 
    subsettest = []
    closedtest = []
    for key in supportkeys:
        # print(key)
        # equiSupport = []
        for k,v in equilistgrp.items():
            if(k == key):
                print(v)
                # lrgfreqset = 
                # print(lrgfreqset)
                closedtest.append(equilistgrp[i][0])

                for j in range(len(equilistgrp[i])):
                    if equilistgrp[i][j][0] != lrgfreqset:
                        if(frozenset(equilistgrp[i][j][0]).issubset(lrgfreqset)):
                            subsettest.append(equilistgrp[i][j])
                        else:
                            closedtest.append(equilistgrp[i][j])
    return subsettest, closedtest
       


In [1328]:
# closedSet(equal,quay)
sorted(equal.items())

[(388,
  [((55307, 55315, 55319, 55323), 388),
   ((55307, 55315, 55319, 55323), 388),
   ((55307, 55315, 55319, 55323), 388)]),
 (389, [((55271, 55323, 55351), 389), ((55271, 55323, 55351), 389)]),
 (390, [((55267, 55295, 55351), 390), ((55267, 55295, 55351), 390)]),
 (391,
  [((55287, 55295, 55315, 55319), 391),
   ((55287, 55295, 55315, 55319), 391),
   ((55287, 55295, 55315, 55319), 391),
   ((55287, 55295, 55315, 55319), 391),
   ((55287, 55295, 55315, 55319), 391)]),
 (392, [((55339, 55343), 392)]),
 (393,
  [((55267, 55271, 55315, 55323), 393), ((55267, 55271, 55315, 55323), 393)]),
 (394, [((55543, 55547), 394), ((55543, 55547), 394), ((55543, 55547), 394)]),
 (395, [((228795,), 395)]),
 (396,
  [((55295, 55319, 55323, 55351), 396),
   ((55295, 55319, 55323, 55351), 396),
   ((55295, 55319, 55323, 55351), 396)]),
 (397,
  [((55287, 55319, 55323, 55351), 397),
   ((55287, 55319, 55323, 55351), 397),
   ((55287, 55319, 55323, 55351), 397),
   ((55287, 55319, 55323, 55351), 397)])

In [1204]:
list(equal.items())
# closedSet(list(ch.items()),sk)
for i,o in equal.items():
    # print(o)
    for k in o:
        print(k[0])

67, 55271, 55295)
(55267, 55271, 55315)
(55267, 55271, 55319)
(55267, 55271, 55351)
(55267, 55291, 55295)
(55267, 55291, 55315)
(55267, 55291, 55319)
(55267, 55295, 55315)
(55267, 55295, 55319)
(55267, 55295, 55351)
(55267, 55315, 55319)
(55267, 55315, 55351)
(55267, 55319, 55351)
(55271, 55291, 55295)
(55271, 55291, 55315)
(55271, 55291, 55319)
(55271, 55295, 55315)
(55271, 55295, 55319)
(55271, 55315, 55319)
(55271, 55319, 55351)
(55291, 55295, 55315)
(55291, 55295, 55319)
(55291, 55315, 55319)
(55291, 55319, 55351)
(55295, 55315, 55319)
(55295, 55315, 55351)
(55295, 55319, 55351)
(55315, 55319, 55351)
(55267, 55315, 55323)
(55267, 55271, 55323)
(55267, 55287, 55323)
(55267, 55287, 55351)
(55267, 55323, 55351)
(55271, 55323, 55351)
(55287, 55323, 55351)
(55315, 55323, 55327)
(55315, 55323, 55351)
(55295, 55319, 55323)
(55319, 55323, 55327)
(55267, 55287, 55315)
(55267, 55287, 55319)
(55287, 55315, 55319)
(55287, 55315, 55351)
(55287, 55319, 55351)
(55287, 55291, 55319)
(55287, 55291,